# 003_Llam3 실습


## 패키지 설치하기
   - accelerate 패키지: Hugging Face 에서 제공하는 패키지로 PyTorch 모델을 간략하고 효율적으로 처리할 수 있음.*이탤릭체 텍스트*


In [ ]:
!pip install accelerate

## 토큰 설정하기

In [ ]:
import os
os.environ['HF_TOKEN'] = "발급 받은 토큰을 입력 하기"


## 패키지 불러오기

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


## 토큰 및 모델 다운로드하기


In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## LLM 응답 함수

In [ ]:
def generate_response(system_message, user_message):
  messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
  ]

  input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt").to(model.device)

  terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

## LLM 응답 함수 테스트 하기

In [ ]:
!wget https://raw.githubusercontent.com/jskimn/aiservice_lesson/main/003_llama3/NvidiaSmiMonitor.py
from NvidiaSmiMonitor import NvidiaSmiMonitor
import sys
import time

monitor = NvidiaSmiMonitor(interval=60)  # 2초마다 실행
monitor.start()

# 10초 동안 실행 후 종료
#time.sleep(10)


In [ ]:
response = generate_response(system_message="",
                              user_message="한국어로 대답해줘. 배화여대에 대해서 말해줘")
print(response)


monitor.stop()
monitor.join()
print("monitor---------종료")